In [1]:
import pandas as pd

# Load the datasets
df_hr = pd.read_csv('hr_terminations.csv')
df_app = pd.read_csv('app_abc_users.csv')

# Convert date columns to actual datetime objects immediately
df_hr['Term_Date'] = pd.to_datetime(df_hr['Term_Date'])
df_app['Last_Login'] = pd.to_datetime(df_app['Last_Login'])

print(f"HR Records: {len(df_hr)}")
print(f"App Records: {len(df_app)}")
df_hr.head()

FileNotFoundError: [Errno 2] No such file or directory: 'hr_terminations.csv'

In [ ]:
# Strip whitespace from IDs and Names to prevent 'false negatives'
df_hr['Employee_ID'] = df_hr['Employee_ID'].str.strip()
df_app['User_ID'] = df_app['User_ID'].str.strip()

# Standardizing names for easier visual review later
df_hr['Name'] = df_hr['Name'].str.strip().str.title()
df_app['Full_Name'] = df_app['Full_Name'].str.strip().str.title()

print("Data cleaning complete.")

In [ ]:
# We join on the ID. 
# We use 'left' because we only care about people on the termination list.
audit_merge = pd.merge(
    df_hr, 
    df_app, 
    left_on='Employee_ID', 
    right_on='User_ID', 
    how='left'
)

# Display the merged table
audit_merge

In [ ]:
# 1. Identify Terminated but still 'Active' in Application
active_leavers = audit_merge[audit_merge['Account_Status'] == 'Active'].copy()

# 2. Identify Logins occurring AFTER termination date
# This is a critical security finding indicating potential account misuse
post_term_logins = audit_merge[audit_merge['Last_Login'] > audit_merge['Term_Date']].copy()

print(f"Finding 1: {len(active_leavers)} users still marked as 'Active'")
print(f"Finding 2: {len(post_term_logins)} users logged in after termination")

In [ ]:
# Create a summary report
with pd.ExcelWriter('Termination_Audit_Report.xlsx') as writer:
    active_leavers.to_excel(writer, sheet_name='Active_Leavers', index=False)
    post_term_logins.to_excel(writer, sheet_name='Post_Term_Logins', index=False)
    audit_merge.to_excel(writer, sheet_name='Full_Traceability_Matrix', index=False)

print("Audit Report Exported: Termination_Audit_Report.xlsx")